In [31]:
import os, argparse, datetime, time, wget, json
import re
import pandas as pd
import sentencepiece as spm
from tqdm.notebook import tqdm
from tqdm import trange

# 학습데이터
## 한국 위키피디아
### 한국 위키피디아 csv 파일 다운로드
* 위키파싱은 wikiextractor의 WikiExtractor.py를 사용 했습니다.

In [36]:
SEPARATOR = u"\u241D"


""" wiki file 목록을 읽어들임 """
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)


""" 여러줄띄기(\n\n...) 한줄띄기로(\n) 변경 """
def trim_text(line):
    try:
        data = json.loads(line)
        text = data["text"]
        value = list(filter(lambda x: len(x) > 0, text.split('\n')))
        data["text"] = "\n".join(value)
        return data
    except:
        return None


""" csv 파일을 제외한 나머지 파일 삭제 """
def del_garbage(dirname):
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            del_garbage(filepath)
            os.rmdir(filepath)
        else:
            if not filename.endswith(".csv"):
                os.remove(filepath)

In [38]:
args_output = 'kowiki'

# wiki를 저장할 폴더 생성
if not os.path.isdir(args_output):
    os.makedirs(args_output)
        
filename = wget.download("https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2", args_output)
os.system(f"python WikiExtractor.py -o {args_output} --json {filename}")

# text 여러줄 띄기를 한줄 띄기로 합침
dataset = []
filenames = list_wiki(args_output)
for filename in filenames:
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                text_line = trim_text(line)
                if text_line is not None:
                    dataset.append(text_line)
    
# 자장파일 결정
now = datetime.datetime.now().strftime("%Y%m%d")
output = f"{args_output}/kowiki_{now}.csv"

# 위키저장 (csv)
if 0 < len(dataset):
    df = pd.DataFrame(data=dataset)
    df.to_csv(output, sep=SEPARATOR, index=False)
    
# 파일삭제
del_garbage(args_output)

# SentencePiece
## SentencePiece Base Model
### 위키 데이터 포맷팅
* 한국 위키데이터 csv 파일을 학습에 사용 할 수 있도록 포맷팅 및 txt 파일로 변환

In [2]:
in_file = "./Data/kowiki_20200715.csv"
out_file = "./Data/kowiki.txt"
SEPARATOR = u"\u241D"
df = pd.read_csv(in_file, sep=SEPARATOR, engine="python")
with open(out_file, "w") as f:
  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
    f.write("\n\n\n\n") # 구분자
len(df)

502158

### SentencePiece 학습
* 학습 데이터로 포맷팅 된 txt 파일은 sentencePiece 학습에 사용되며, prefix는 kowiki_30000, vocab_size는 30,000으로 임의 설정

In [3]:
corpus = "./Data/kowiki.txt"
prefix = "kowiki_30000"
vocab_size = 30000
# prefix = "kowiki_50000"
# vocab_size = 50000
# prefix = "kowiki_100000"
# vocab_size = 100000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

### SentencePiece Base Model Test
* 학습된 kowiki_30000.model을 load
* test_sentences.xlsx에 있는 테스트 문장들을 tokenizing 후 결과 확인
    * 특수 문자(,.-()*&^%$#@!) 제거 후 프로세스 진행 

In [40]:
vocab_file = "./Model/kowiki_30000.model"
sp = spm.SentencePieceProcessor()
sp.load(vocab_file)

True

In [41]:
def pre_sentence(row):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', row)
    return text

def tokenize(row):
    tokens = sp.encode_as_pieces(row['test_sentence'])
    sent = 'V'.join(tokens)
    sent = sent.replace('▁', ' ')
    return sent

df = pd.read_excel('./UserData/test_sentences.xlsx')
df['test_sentence'] = df['test_sentence'].apply(pre_sentence)
df['sentence'] = df.apply(tokenize, axis=1)
df

,test_sentence,sentence
0,포스코케미칼 차세대 배터리 양극재 개발 매일경제,포V스코V케V미V칼V 차세대V 배터리V 양극V재V 개발V 매일V경제
1,더블유게임즈의 전신은 2012년 4월 온라인 게임 및 개발 서비스를 목적으로 설립된...,더블V유V게임V즈의V 전V신은V 2012V년V 4V월V 온라인V 게임V 및V 개발...
2,손태승 회장의 디지털 혁신 우리금융그룹 1위 도약 이끈다,손V태V승V 회V장의V 디지털V 혁신V 우리V금융V그룹V 1V위V 도V약V 이끈V다
3,HDC현대산업개발 9000억원대 서창김포 고속도로 건설 우선협상대상자 선정,HDVCV현대V산업V개발V 9V000V억원V대V 서V창V김V포V 고속도로V 건설V...
4,애경산업 케라시스 홍삼농축액 담은 헤어케어 선보여,애V경V산업V 케V라V시스V 홍V삼V농V축V액V 담은V 헤어V케V어V 선보V여
5,GC녹십자와 셀트리온 중심으로 준비중예상보다 늦어질듯 국내에서 개발 중인 신종 코로...,GVCV녹V십V자와V 셀V트리V온V 중심으로V 준비V중V예V상V보다V 늦V어V질V...
6,쿠쿠 아이스텐에스 정수기 판매량 126 급등,쿠V쿠V 아이스V텐V에스V 정수V기V 판매량V 12V6V 급V등
7,BGF리테일 전국 CU 매장 5천여곳 폭염쉼터로 24시간 개방,BVGVFV리V테일V 전국V CVUV 매장V 5V천여V곳V 폭V염V쉼V터로V 24...
8,카카오 카카오페이로 해외 직구도 한다,카카오V 카카오V페V이로V 해외V 직V구V도V 한다
9,오리온 케이준 눈을감자’ 출시,오리온V 케이V준V 눈을V감V자V’V 출시


## Add words without training
### Environments setting
* https://github.com/google/sentencepiece#c-from-source
    * 위와 같이 sentencePiece 설치
    * spm_export_vocab --model=xxx.model --output_format=vocab --output=xxx.vocab
* sentencepiece_model_pb2.py 가져오기

In [10]:
!wget "https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_model_pb2.py"

--2020-07-20 11:23:17--  https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_model_pb2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.228.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30433 (30K) [text/plain]
Saving to: ‘sentencepiece_model_pb2.py’

sentencepiece_model 100%[===================>]  29.72K  --.-KB/s    in 0.04s   

2020-07-20 11:23:17 (698 KB/s) - ‘sentencepiece_model_pb2.py’ saved [30433/30433]



### Words add
* sentencepiece model proto를 통해 새로운 단어를 학습 과정 없이 model에 추가.

In [46]:
import sentencepiece_model_pb2 as model

m = model.ModelProto()
m.ParseFromString(open(vocab_file, 'rb').read())

761836

In [47]:
piece_df = pd.read_excel('./UserData/user_dictionary.xlsx')
add_pieces = piece_df['add_keywords'].to_list()
len(add_pieces)

34

In [48]:
for piece in tqdm(add_pieces, total=len(add_pieces)):
    tokens = sp.encode_as_pieces(piece)

    # 이미 사전에 등록된 키워드인 경우 별도의 추가 없이 패스
    if '▁' + piece == tokens[0]:
        continue
    
    # proto burf
    word = '▁' + piece
    piece_obj = m.pieces.add()
    piece_obj.piece = word
    piece_obj.score = -10
    piece_obj.type = m.SentencePiece.USER_DEFINED
    
    # 추가를 진행 시 vacab 사이즈가 기존 보다 클 경우 에러 발생
    # 에러가 발생하기 때문에 해당 단어의 첫 sub word를 교체하는 방식으로 진행
    for p in m.pieces:
        if p.piece == tokens[0]:
            print("origin piece - {0}, change piece - {1}".format(p.piece, word))
            p = piece

with open('./Model/new_domain.model', 'wb') as f:
    f.write(m.SerializeToString())

origin piece - ▁포, change piece - ▁포스코케미칼
origin piece - ▁더블, change piece - ▁더블유게임즈
origin piece - ▁우리, change piece - ▁우리금융지주
origin piece - ▁HD, change piece - ▁HDC현대산업개발
origin piece - ▁애, change piece - ▁애경산업
origin piece - ▁셀, change piece - ▁셀트리온
origin piece - ▁쿠, change piece - ▁쿠쿠홈시스
origin piece - ▁B, change piece - ▁BGF리테일
origin piece - ▁넷, change piece - ▁넷마블
origin piece - ▁삼성, change piece - ▁삼성바이오로직스
origin piece - ▁L, change piece - ▁LIG넥스원
origin piece - ▁이노, change piece - ▁이노션
origin piece - ▁삼성, change piece - ▁삼성물산
origin piece - ▁만, change piece - ▁만도
origin piece - ▁코스, change piece - ▁코스맥스
origin piece - ▁종, change piece - ▁종근당
origin piece - ▁현, change piece - ▁현대로템
origin piece - ▁동아, change piece - ▁동아에스티
origin piece - ▁한국, change piece - ▁한국콜마
origin piece - ▁신세계, change piece - ▁신세계인터내셔날
origin piece - ▁롯데, change piece - ▁롯데하이마트
origin piece - ▁이, change piece - ▁이마트
origin piece - ▁B, change piece - ▁BNK금융지주
origin piece - ▁현대, change piece - ▁현대위아
ori

In [49]:
!spm_export_vocab --model=./Model/new_domain.model --output_format=vocab --output=./Model/new_domain.vocab

### SentencePiece Test
* 추가된 키워드가 정상적으로 tokenizing 되는지 테스트
* 테스트 결과 기존의 모델과 비교

In [21]:
vocab_file = "./new_domain.model"
sp = spm.SentencePieceProcessor()
sp.load(vocab_file)

True

In [22]:
def pre_sentence(row):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', row)
    return text

def tokenize(row):
    tokens = sp.encode_as_pieces(row['test_sentence'])
    sent = 'V'.join(tokens)
    sent = sent.replace('▁', ' ')
    return sent

domain_df = pd.read_excel('./UserData/test_sentences.xlsx')
domain_df['test_sentence'] = domain_df['test_sentence'].apply(pre_sentence)
domain_df['domain_sentence'] = domain_df.apply(tokenize, axis=1)
domain_df

,test_sentence,domain_sentence
0,포스코케미칼 차세대 배터리 양극재 개발 매일경제,포스코케미칼V 차세대V 배터리V 양극V재V 개발V 매일V경제
1,더블유게임즈의 전신은 2012년 4월 온라인 게임 및 개발 서비스를 목적으로 설립된...,더블유게임즈V의V 전V신은V 2012V년V 4V월V 온라인V 게임V 및V 개발V ...
2,손태승 회장의 디지털 혁신 우리금융그룹 1위 도약 이끈다,손V태V승V 회V장의V 디지털V 혁신V 우리V금융V그룹V 1V위V 도V약V 이끈V다
3,HDC현대산업개발 9000억원대 서창김포 고속도로 건설 우선협상대상자 선정,HDC현대산업개발V 9V000V억원V대V 서V창V김V포V 고속도로V 건설V 우선V...
4,애경산업 케라시스 홍삼농축액 담은 헤어케어 선보여,애경산업V 케V라V시스V 홍V삼V농V축V액V 담은V 헤어V케V어V 선보V여
5,GC녹십자와 셀트리온 중심으로 준비중예상보다 늦어질듯 국내에서 개발 중인 신종 코로...,GVCV녹V십V자와V 셀트리온V 중심으로V 준비V중V예V상V보다V 늦V어V질V듯V...
6,쿠쿠 아이스텐에스 정수기 판매량 126 급등,쿠V쿠V 아이스V텐V에스V 정수V기V 판매량V 12V6V 급V등
7,BGF리테일 전국 CU 매장 5천여곳 폭염쉼터로 24시간 개방,BGF리테일V 전국V CVUV 매장V 5V천여V곳V 폭V염V쉼V터로V 24V시간V 개방
8,카카오 카카오페이로 해외 직구도 한다,카카오V 카카오V페V이로V 해외V 직V구V도V 한다
9,오리온 케이준 눈을감자’ 출시,오리온V 케이V준V 눈을V감V자V’V 출시


In [23]:
pd.merge(domain_df, df)

,test_sentence,domain_sentence,sentence
0,포스코케미칼 차세대 배터리 양극재 개발 매일경제,포스코케미칼V 차세대V 배터리V 양극V재V 개발V 매일V경제,포V스코V케V미V칼V 차세대V 배터리V 양극V재V 개발V 매일V경제
1,더블유게임즈의 전신은 2012년 4월 온라인 게임 및 개발 서비스를 목적으로 설립된...,더블유게임즈V의V 전V신은V 2012V년V 4V월V 온라인V 게임V 및V 개발V ...,더블V유V게임V즈의V 전V신은V 2012V년V 4V월V 온라인V 게임V 및V 개발...
2,손태승 회장의 디지털 혁신 우리금융그룹 1위 도약 이끈다,손V태V승V 회V장의V 디지털V 혁신V 우리V금융V그룹V 1V위V 도V약V 이끈V다,손V태V승V 회V장의V 디지털V 혁신V 우리V금융V그룹V 1V위V 도V약V 이끈V다
3,HDC현대산업개발 9000억원대 서창김포 고속도로 건설 우선협상대상자 선정,HDC현대산업개발V 9V000V억원V대V 서V창V김V포V 고속도로V 건설V 우선V...,HDVCV현대V산업V개발V 9V000V억원V대V 서V창V김V포V 고속도로V 건설V...
4,애경산업 케라시스 홍삼농축액 담은 헤어케어 선보여,애경산업V 케V라V시스V 홍V삼V농V축V액V 담은V 헤어V케V어V 선보V여,애V경V산업V 케V라V시스V 홍V삼V농V축V액V 담은V 헤어V케V어V 선보V여
5,GC녹십자와 셀트리온 중심으로 준비중예상보다 늦어질듯 국내에서 개발 중인 신종 코로...,GVCV녹V십V자와V 셀트리온V 중심으로V 준비V중V예V상V보다V 늦V어V질V듯V...,GVCV녹V십V자와V 셀V트리V온V 중심으로V 준비V중V예V상V보다V 늦V어V질V...
6,쿠쿠 아이스텐에스 정수기 판매량 126 급등,쿠V쿠V 아이스V텐V에스V 정수V기V 판매량V 12V6V 급V등,쿠V쿠V 아이스V텐V에스V 정수V기V 판매량V 12V6V 급V등
7,BGF리테일 전국 CU 매장 5천여곳 폭염쉼터로 24시간 개방,BGF리테일V 전국V CVUV 매장V 5V천여V곳V 폭V염V쉼V터로V 24V시간V 개방,BVGVFV리V테일V 전국V CVUV 매장V 5V천여V곳V 폭V염V쉼V터로V 24...
8,카카오 카카오페이로 해외 직구도 한다,카카오V 카카오V페V이로V 해외V 직V구V도V 한다,카카오V 카카오V페V이로V 해외V 직V구V도V 한다
9,오리온 케이준 눈을감자’ 출시,오리온V 케이V준V 눈을V감V자V’V 출시,오리온V 케이V준V 눈을V감V자V’V 출시
